## 问题二

In [103]:
import pandas as pd
import numpy as np

In [104]:
# 加载历史货物量数据
historical_volumes = pd.read_csv("附件1.csv", encoding="gb2312")

# 加载历史关联数据
historical_links = pd.read_csv("附件3.csv", encoding="gb2312")

# 加载未来的分拣中心关联数据
future_links = pd.read_csv("附件4.csv", encoding="gb2312")

In [105]:
# # 查看历史货物量数据
# print(historical_volumes.head())
# print(historical_volumes.info())

# # 查看历史关联数据
# print(historical_links.head())
# print(historical_links.info())

# # 查看未来关联数据
# print(future_links.head())
# print(future_links.info())

In [106]:
# 转换日期格式
historical_volumes['日期'] = pd.to_datetime(historical_volumes['日期'], format='%Y/%m/%d')

# 检查是否有缺失值，并考虑填充或删除
print(historical_volumes.isnull().sum())
print(historical_links.isnull().sum())
print(future_links.isnull().sum())

# 如果有必要，填充缺失值，这里假设没有缺失值
# historical_volumes.fillna(method='ffill', inplace=True)

分拣中心    0
日期      0
货量      0
dtype: int64
始发分拣中心    0
到达分拣中心    0
货量        0
dtype: int64
始发分拣中心    0
到达分拣中心    0
dtype: int64


In [107]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [108]:
# 选择一个分拣中心的数据进行演示
sc_data = historical_volumes[historical_volumes['分拣中心'] == 'SC48']
sc_data

,分拣中心,日期,货量
0,SC48,2023-09-05,723
1,SC48,2023-10-24,2092
2,SC48,2023-08-05,754
3,SC48,2023-11-16,754
4,SC48,2023-11-09,771
...,...,...,...
117,SC48,2023-09-25,785
118,SC48,2023-08-01,685
119,SC48,2023-08-18,710
120,SC48,2023-09-04,807


In [109]:
# 数据标准化
scaler = MinMaxScaler()
sc_data_scaled = scaler.fit_transform(sc_data["货量"].values.reshape(-1, 1))

sc_data_scaled

array([[0.06040503],
       [0.53840782],
       [0.07122905],
       [0.07122905],
       [0.0771648 ],
       [0.04958101],
       [0.09392458],
       [0.03247207],
       [0.01396648],
       [0.03421788],
       [0.04643855],
       [0.96473464],
       [0.04993017],
       [0.05726257],
       [0.04958101],
       [0.10265363],
       [0.08170391],
       [0.05167598],
       [0.09951117],
       [0.02199721],
       [0.07856145],
       [0.04888268],
       [0.05132682],
       [1.        ],
       [0.08065642],
       [0.04643855],
       [0.07367318],
       [0.05377095],
       [0.04888268],
       [0.2224162 ],
       [0.05796089],
       [0.05761173],
       [0.03037709],
       [0.06110335],
       [0.04888268],
       [0.04294693],
       [0.05377095],
       [0.06145251],
       [0.0652933 ],
       [0.0247905 ],
       [0.05377095],
       [0.09706704],
       [0.13617318],
       [0.06738827],
       [0.09567039],
       [0.0652933 ],
       [0.04958101],
       [0.070

In [110]:
# 创建时间序列数据集
def create_dataset(data, time_step=1):
    X, Y = [], []
    for i in range(len(data) - time_step - 1):
        a = data[i : (i + time_step), 0]
        X.append(a)
        Y.append(data[i + time_step, 0])
    return np.array(X), np.array(Y)

# 使用所有分拣中心的数据
all_data_scaled = scaler.fit_transform(historical_volumes["货量"].values.reshape(-1, 1))

time_step = 3
X, y = create_dataset(all_data_scaled, time_step)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Reshape input to be [samples, time steps, features]
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [111]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=64, verbose=1)

model.save('sc48_lstm_model.h5')

Epoch 1/100


/home/yinmo19/Mathor_cup_2024/venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


87/87 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0153 - val_loss: 0.0028
Epoch 2/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0033 - val_loss: 0.0027
Epoch 3/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0030 - val_loss: 0.0026
Epoch 4/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0032 - val_loss: 0.0027
Epoch 5/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0029 - val_loss: 0.0025
Epoch 6/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0031 - val_loss: 0.0027
Epoch 7/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0031 - val_loss: 0.0025
Epoch 8/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0028 - val_loss: 0.0024
Epoch 9/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0028 - val_loss: 0.0026
Epoch 10/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0033 - val_loss: 0.0024
Epoch 11/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0028 - val_loss: 0.0023
Epoch 12/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0026 - val_l

In [112]:
import pandas as pd
import torch
from torch_geometric.data import Data
import networkx as nx
from sklearn.preprocessing import MinMaxScaler
from torch_geometric.utils import from_networkx

In [113]:
# 假设 historical_links 已经加载
# 转换数据为 networkx 图
G = nx.from_pandas_edgelist(
    historical_links,
    source="始发分拣中心",
    target="到达分拣中心",
    edge_attr="货量",
    create_using=nx.DiGraph(),
)

# 为了能够使用 GCN，我们需要一个无向图
G = G.to_undirected()

# 将 networkx 图转换为 torch_geometric 数据
# 首先为每个节点赋予初始特征（可以是度数等）
for node in G.nodes():
    G.nodes[node]["x"] = [float(G.degree[node])]

data = from_networkx(G)
data.edge_attr = torch.tensor(
    list(nx.get_edge_attributes(G, "货量").values()), dtype=torch.float
).unsqueeze(1)

# 标准化边属性
scaler = MinMaxScaler()
data.edge_attr = torch.tensor(scaler.fit_transform(data.edge_attr), dtype=torch.float)

In [114]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader

class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, 1)  # Assuming each edge will have one output feature: the cargo volume

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        return self.conv3(x, edge_index)


In [115]:

model = GCN(num_node_features=1, hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()


In [116]:
import pandas as pd
import torch
from torch_geometric.data import Data

# 加载数据
historical_volumes = pd.read_csv("附件1.csv", encoding="gb2312")
historical_links = pd.read_csv("附件3.csv", encoding="gb2312")
future_links = pd.read_csv("附件4.csv", encoding="gb2312")

# 创建节点列表
all_nodes = pd.unique(
    historical_links[["始发分拣中心", "到达分拣中心"]].values.ravel("K")
)
node_indices = {node: idx for idx, node in enumerate(all_nodes)}

# 创建边索引
edge_index = torch.tensor(
    [
        historical_links["始发分拣中心"].map(node_indices).values,
        historical_links["到达分拣中心"].map(node_indices).values,
    ],
    dtype=torch.long,
)

# 节点特征：使用节点的总货物量作为特征
node_features = torch.zeros(len(all_nodes), 1)
for node in all_nodes:
    total_volume = historical_volumes[historical_volumes["分拣中心"] == node][
        "货量"
    ].sum()
    node_features[node_indices[node], 0] = total_volume

# 目标值和训练掩码
data_y = torch.zeros(len(all_nodes), 1)
train_mask = torch.zeros(len(all_nodes), dtype=torch.bool)
# 示例：我们使用历史数据中的货物量总和最高的节点作为训练节点
top_nodes = historical_volumes.groupby("分拣中心")["货量"].sum().nlargest(10).index
for node in top_nodes:
    if node in node_indices:
        data_y[node_indices[node]] = historical_volumes[
            historical_volumes["分拣中心"] == node
        ]["货量"].sum()
        train_mask[node_indices[node]] = True

# 创建Data对象
data = Data(x=node_features, edge_index=edge_index, y=data_y, train_mask=train_mask)

In [117]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler_x = StandardScaler()
data.x = torch.tensor(scaler_x.fit_transform(data.x), dtype=torch.float)

scaler_y = StandardScaler()
data.y = torch.tensor(scaler_y.fit_transform(data.y.reshape(-1, 1)), dtype=torch.float).squeeze()
# 假设data已经准备好
dataset = [data]  # 如果有多个图，可以添加到列表中
loader = DataLoader(dataset, batch_size=1, shuffle=True)


/home/yinmo19/Mathor_cup_2024/venv/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [118]:
# 假设 future_links 已经加载
future_edges = pd.read_csv("附件4.csv", encoding="gb2312")

# 更新边索引
future_edge_index = torch.tensor([
    future_edges['始发分拣中心'].map(node_indices).values,
    future_edges['到达分拣中心'].map(node_indices).values
], dtype=torch.long)

# 使用更新后的边索引创建新的图数据
future_data = Data(x=data.x.clone(), edge_index=future_edge_index, y=data.y.clone(), train_mask=data.train_mask.clone())


In [119]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out, data.edge_attr)  # 假设 data.edge_attr 是实际货物流量
    loss.backward()
    optimizer.step()
    return loss.item()

# 训练模型
for epoch in range(200):
    loss = train()
    print(f'Epoch {epoch+1}: Loss {loss:.4f}')
    
# 进行预测
model.eval()
with torch.no_grad():
    predictions = model(data.x, data.edge_index)
    predictions = predictions.squeeze()

# 打印预测的货物流量
edges = data.edge_index.t().cpu().numpy()
for i, edge in enumerate(edges):
    print(f"SC{edge[0]} to SC{edge[1]}: {predictions[i].item():.2f}")

AttributeError: 'NoneType' object has no attribute 'size'

In [97]:
def predict(data):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        predicted_volumes = out.squeeze()
    return predicted_volumes

# 进行预测
predicted_volumes = predict(future_data)

In [98]:
def predict_daily_volumes(model, data, days=30):
    model.eval()
    daily_volumes = []
    with torch.no_grad():
        for day in range(days):
            out = model(data.x, data.edge_index)
            predicted_volumes = out.squeeze()
            # 确保predicted_volumes是二维的，以符合scaler_y的要求
            predicted_volumes = predicted_volumes.view(-1, 1)  # 从1D转换为2D，如果需要的话
            predicted_volumes = scaler_y.inverse_transform(predicted_volumes.numpy()).flatten()
            daily_volumes.append(predicted_volumes)
    return daily_volumes

# 使用函数进行预测
future_daily_volumes = predict_daily_volumes(model, future_data, 30)

# 输出预测结果
for day, volumes in enumerate(future_daily_volumes, 1):
    print(f"Day {day}:")
    for i, volume in enumerate(volumes):
        print(f"  {all_nodes[i]}: {volume}")

Day 1:
  SC22: 1950545.75
  SC18: 2087647.875
  SC7: 4518155.0
  SC15: 2719910.5
  SC52: 2299113.75
  SC2: 3000550.5
  SC28: 2527367.0
  SC19: 1921044.5
  SC44: 3915013.25
  SC23: 2387723.0
  SC30: 2250427.75
  SC36: 2561541.5
  SC4: 3739726.0
  SC24: 2111710.25
  SC8: 4246687.0
  SC63: 2402423.5
  SC51: 3102367.75
  SC55: 2301866.0
  SC43: 2306359.0
  SC48: 2554926.5
  SC38: 2276400.0
  SC32: 2366624.5
  SC21: 2426011.0
  SC12: 2230772.5
  SC27: 2253245.5
  SC16: 2404737.25
  SC29: 2033294.875
  SC56: 2462483.0
  SC37: 2551911.5
  SC54: 1763179.25
  SC58: 2549785.0
  SC1: 3961780.0
  SC25: 4288059.5
  SC5: 4148125.0
  SC66: 2550920.0
  SC60: 3292059.5
  SC68: 2509217.5
  SC10: 3736438.5
  SC61: 2544329.5
  SC26: 2240065.5
  SC41: 2453746.5
  SC39: 2398380.5
  SC3: 2059235.25
  SC9: 2310365.5
  SC53: 2085295.875
  SC34: 1805969.5
  SC31: 2310365.5
  SC35: 2059235.25
  SC47: 4894072.0
  SC6: 3614447.75
  SC57: 3316262.0
Day 2:
  SC22: 1950545.75
  SC18: 2087647.875
  SC7: 4518155.0
  SC

In [99]:
# 检查 scaler_y 的 scale_ 和 mean_ 属性
print("Scale:", scaler_y.scale_)
print("Mean:", scaler_y.mean_)

Scale: [1947743.50687798]
Mean: [933516.]


In [100]:
print("Original y values:", historical_volumes.groupby("分拣中心")["货量"].sum().nlargest(10))
print("Standardized y values:", data.y[data.train_mask])

Original y values: 分拣中心
SC60    6975702
SC47    5603084
SC1     5445878
SC25    5151430
SC8     5019169
SC7     4738272
SC53    3944687
SC10    3806786
SC6     3510835
SC35    3413473
Name: 货量, dtype: int64
Standardized y values: tensor([1.9534, 2.0976, 2.3167, 2.1655, 3.1021, 1.4752, 1.5460, 1.2732, 2.3974,
        1.3232])


In [101]:
def predict_daily_volumes(model, data, days=30):
    model.eval()
    daily_volumes = []
    with torch.no_grad():
        for day in range(days):
            out = model(data.x, data.edge_index)
            predicted_volumes = out.squeeze()
            print("Model raw output:", predicted_volumes)  # 打印原始模型输出观察
            predicted_volumes = scaler_y.inverse_transform(predicted_volumes.view(-1, 1)).flatten()
            daily_volumes.append(predicted_volumes)
    return daily_volumes

# 调用函数进行预测
future_daily_volumes = predict_daily_volumes(model, future_data, 30)

Model raw output: tensor([0.5222, 0.5925, 1.8404, 0.9172, 0.7011, 1.0612, 0.8183, 0.5070, 1.5307,
        0.7466, 0.6761, 0.8359, 1.4407, 0.6049, 1.7010, 0.7542, 1.1135, 0.7025,
        0.7048, 0.8325, 0.6895, 0.7358, 0.7663, 0.6660, 0.6776, 0.7553, 0.5646,
        0.7850, 0.8309, 0.4260, 0.8298, 1.5548, 1.7223, 1.6504, 0.8304, 1.2109,
        0.8090, 1.4391, 0.8270, 0.6708, 0.7805, 0.7521, 0.5780, 0.7069, 0.5913,
        0.4479, 0.7069, 0.5780, 2.0334, 1.3764, 1.2233])
Model raw output: tensor([0.5222, 0.5925, 1.8404, 0.9172, 0.7011, 1.0612, 0.8183, 0.5070, 1.5307,
        0.7466, 0.6761, 0.8359, 1.4407, 0.6049, 1.7010, 0.7542, 1.1135, 0.7025,
        0.7048, 0.8325, 0.6895, 0.7358, 0.7663, 0.6660, 0.6776, 0.7553, 0.5646,
        0.7850, 0.8309, 0.4260, 0.8298, 1.5548, 1.7223, 1.6504, 0.8304, 1.2109,
        0.8090, 1.4391, 0.8270, 0.6708, 0.7805, 0.7521, 0.5780, 0.7069, 0.5913,
        0.4479, 0.7069, 0.5780, 2.0334, 1.3764, 1.2233])
Model raw output: tensor([0.5222, 0.5925, 1.8404, 

In [102]:
from sklearn.preprocessing import MinMaxScaler

# 节点特征和目标值的 MinMax 缩放
minmax_scaler_x = MinMaxScaler()
data.x = torch.tensor(minmax_scaler_x.fit_transform(data.x), dtype=torch.float)

minmax_scaler_y = MinMaxScaler()
data.y = torch.tensor(minmax_scaler_y.fit_transform(data.y.reshape(-1, 1)), dtype=torch.float).squeeze()

# 在预测函数中使用 MinMaxScaler 进行逆变换
def predict_daily_volumes(model, data, days=30):
    model.eval()
    daily_volumes = []
    with torch.no_grad():
        for day in range(days):
            out = model(data.x, data.edge_index)
            predicted_volumes = out.squeeze()
            # 使用 MinMaxScaler 逆变换
            predicted_volumes = minmax_scaler_y.inverse_transform(predicted_volumes.view(-1, 1)).flatten()
            daily_volumes.append(predicted_volumes)
    return daily_volumes

# 调用函数进行预测
future_daily_volumes = predict_daily_volumes(model, future_data, 30)

# 输出预测结果
for day, volumes in enumerate(future_daily_volumes, 1):
    print(f"Day {day}:")
    for i, volume in enumerate(volumes):
        print(f"  {all_nodes[i]}: {volume:.2f}")


Day 1:
  SC22: 1.39
  SC18: 1.64
  SC7: 6.11
  SC15: 2.81
  SC52: 2.03
  SC2: 3.32
  SC28: 2.45
  SC19: 1.34
  SC44: 5.00
  SC23: 2.19
  SC30: 1.94
  SC36: 2.51
  SC4: 4.68
  SC24: 1.69
  SC8: 5.61
  SC63: 2.22
  SC51: 3.51
  SC55: 2.04
  SC43: 2.05
  SC48: 2.50
  SC38: 1.99
  SC32: 2.16
  SC21: 2.27
  SC12: 1.91
  SC27: 1.95
  SC16: 2.23
  SC29: 1.54
  SC56: 2.33
  SC37: 2.50
  SC54: 1.05
  SC58: 2.49
  SC1: 5.09
  SC25: 5.69
  SC5: 5.43
  SC66: 2.49
  SC60: 3.86
  SC68: 2.42
  SC10: 4.67
  SC61: 2.48
  SC26: 1.92
  SC41: 2.32
  SC39: 2.21
  SC3: 1.59
  SC9: 2.05
  SC53: 1.64
  SC34: 1.12
  SC31: 2.05
  SC35: 1.59
  SC47: 6.80
  SC6: 4.45
  SC57: 3.90
Day 2:
  SC22: 1.39
  SC18: 1.64
  SC7: 6.11
  SC15: 2.81
  SC52: 2.03
  SC2: 3.32
  SC28: 2.45
  SC19: 1.34
  SC44: 5.00
  SC23: 2.19
  SC30: 1.94
  SC36: 2.51
  SC4: 4.68
  SC24: 1.69
  SC8: 5.61
  SC63: 2.22
  SC51: 3.51
  SC55: 2.04
  SC43: 2.05
  SC48: 2.50
  SC38: 1.99
  SC32: 2.16
  SC21: 2.27
  SC12: 1.91
  SC27: 1.95
  SC16: 2.2